# HashiCorp Vault Demo for KMIP Secrets Engine and MongoDB Client-Side Field Level Encryption (CSFLE) & Queryable Encryption (QE)

With MongoDB’s Client-Side Field Level Encryption (CSFLE) and Queryable Encryption (QE), applications can encrypt sensitive plain text fields in documents prior to transmitting data to the server. This means that data processed by database (in use) will not be in plain text as it’s always encrypted and most importantly still can be queried (QE). 

This demo showcases how MongoDB CSFLE and QE uses HashiCorp Vault's KMIP Secrets Engine to manage encryption keys via KMIP protocol.

Note: MongoDB Community & Enterprise server editions have the same developer features (queries, aggregation, replication, sharding, etc).  MongoDB Enterprise Advanced adds:
- Operational & security features (In-memory storage engine, Auditing, Kerberos & LDAP authentication, Encryption at Rest). https://www.mongodb.com/docs/manual/administration/upgrade-community-to-enterprise/
- Entitlements to use additional tools including Ops Manager, BI Connector, and Enterprise Operator for Kubernetes.

CSFLE is offered on both Community and Enterprise Edition. The only difference is that Enterprise edition can perform automatic encryption based on JSONSchema (which is also used by the mongocryptd library). For the community edition the application will be responsible for performing this encryption. 
- https://www.mongodb.com/docs/manual/core/csfle/reference/compatibility/
- https://www.mongodb.com/docs/manual/core/queryable-encryption/reference/compatibility/

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- MongoDB Enterprise installed on docker (to simulate an external MongoDB)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP KMIP license add-on.  Also note that a Premium license is required if you wish the KMIP listener to scale on the Vault cluster.  For Standard or Plus license, the KMIP listener is only on the leader node.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Make sure Python 3 is installed for the tutorial python scripts to run
brew install python3

In [ ]:
# Install python dependencies for the CSFLE/QE python script samples
python3 -m pip install pymongo
python3 -m pip install 'pymongo[encryption]'


In [ ]:
# Optional: To upgrade, uninstall current version
python3 -m pip uninstall -y pymongo


In [ ]:
# Optional: Check pymongo version that you are using
python3 -m pip list | grep pymongo

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose both the Vault API and the KMIP ports to the host machine.
# We will be using 5697 instead of 5696 for the KMIP port as the CSFLE/QE samples are already configured for that port
export VAULT_PORT=8200
export VAULT_KMIP_PORT=5697
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_KMIP_PORT}:${VAULT_KMIP_PORT} \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Setting up MongDB Enterprise

Note: Queryable Encryption does not work on a standalone instance.  Needs either a replica set or a sharded cluster.  For this demo, we will be using a single node replica set.
Ref: https://www.mongodb.com/community/forums/t/i-found-an-error-when-trying-to-use-mongodbs-queryable-encryption/188433/4

In [ ]:
# Run MongoDB Enterprise in docker.  Expose the MongoDB port to the host machine. 
export MONGODB_PORT=27017
# Use latest 7.0 MongoDB Enterprise docker image.  This supports both linux/amd64 and linux/arm64
export MONGODB_TAG=7.0-ubuntu2204
# For UBI images
#export MONGODB_TAG=7.0-ubi8

docker run -d --rm -p ${MONGODB_PORT}:${MONGODB_PORT} --name mongodb-enterprise \
mongodb/mongodb-enterprise-server:$MONGODB_TAG mongod --replSet rs0 --port $MONGODB_PORT  \
--bind_ip "0.0.0.0"

In [ ]:
# Initialize the replica set with a default configuration set
# Ref: https://www.mongodb.com/docs/manual/reference/method/rs.initiate/
docker exec -it mongodb-enterprise mongosh --eval "rs.initiate()"

In [ ]:
# View the status of your replica set
docker exec -it mongodb-enterprise mongosh --eval "rs.status()"

# Setup MongoDB CSFLE/QE Sample

This section demonstrates the KMIP secrets engine and how it functions as a KMIP server with MongoDB Client-Side Field Level Encryption (CSFLE) and Queryable Encryption (QE).

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip
- https://www.mongodb.com/developer/products/atlas/hashicorp-vault-kmip-secrets-engine-mongodb/
- https://github.com/mongodb-developer/mongodb-kmip-fle-queryable

In [ ]:
# Clone the tutorial repository for CSFLE/QE samples
git clone https://github.com/mongodb-developer/mongodb-kmip-fle-queryable

## Step 1 - Enable Vault KMIP Engine

In [ ]:
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

In [ ]:
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT default_tls_client_ttl=365d
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d

# Check the KMIP configuration
echo
vault read kmip/config

## Step 2 - Create KMIP Scope and Role in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [ ]:
# Settings for the KMIP scope and role to be created
export KMIP_SVC_NAME=mongodb-svc
export KMIP_FLE_ROLE_NAME=fle

# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# Create FLE Role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/fle tls_client_key_bits=2048 tls_client_key_type=rsa operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP roles have been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_FLE_ROLE_NAME

## Step 3 - Create the KMIP certificates for MongoDB CSFLE/QE

In [ ]:
# Folder that is used by the CSFLE sample to store the KMIP certificates
export SAMPLE_PATH=./mongodb-kmip-fle-queryable/kmip-with-hashicorp-key-vault
export FLE_CERT_PATH=$SAMPLE_PATH/vault/certs/FLE

# Save KMIP CA certificate
vault read -format=json kmip/ca | jq -r .data.ca_pem > $FLE_CERT_PATH/vv-ca.pem    

# Create KMIP client certificates
vault write -format=json kmip/scope/$KMIP_SVC_NAME/role/$KMIP_FLE_ROLE_NAME/credential/generate format=pem > fle_credential.json
jq -r .data.certificate < fle_credential.json > $FLE_CERT_PATH/vv-cert.pem
jq -r .data.private_key < fle_credential.json > $FLE_CERT_PATH/vv-key.pem    
cat $FLE_CERT_PATH/vv-cert.pem $FLE_CERT_PATH/vv-key.pem > $FLE_CERT_PATH/vv-client.pem

# Cleanup temp json
rm fle_credential.json

## Step 4 - Update the CSFLE/QE Sample Connection String

In [ ]:
# Replace this line in the configuration_fle.py file 
#   connection_uri = "mongodb+srv://<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority"
# with this
#   connection_uri = "mongodb://127.0.0.1:27017/admin?replicaSet=rs0&retryWrites=true&directConnection=true"
# Replace the connection string
# Escape "/"" with "\/"" and "&"" with "\&""
sed -i '' 's/mongodb+srv:\/\/<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority/mongodb:\/\/127.0.0.1:27017\/admin?replicaSet=rs0\&retryWrites=true\&directConnection=true/g' $SAMPLE_PATH/configuration_fle.py

# Show updated configuration_fle.py
cat $SAMPLE_PATH/configuration_fle.py

## Step 5 - Test the CSFLE/QE Sample

In [ ]:
# Test the CSFLE sample.  This script demonstrates encrypt and decrypt functions with the HashiCorp Vault KMIP provider.
# Change to the script directory before running as the scripts are hard-code to find the client certs in /vault/certs/FLE
cd $SAMPLE_PATH
python3 vault_encrypt_with_csfle_kmip.py
cd ../..

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Stop Vault container
docker stop vault-enterprise

# Stop MongoDB container
docker stop mongodb-enterprise

# Remove KMIP certificate demo files
rm -rf mongodb-kmip-fle-queryable